# Co-factor swap

<!-- The balance of co-factors within the cells is important 
In the implementated pathway of progesterone nine NADP and one NAD molecules 
In order for the cell to produce one pregesterone molecule through the implementated pathway nine NADPH molecules and one NAD $^+$ molecules are needed. 
 -->



The balance of co-factors within a cell is important to obtain a high theoretical yield of a given product (King, Zachary A., and Adam M. Feist, 2014). The NADP(H) and NAD(H) co-factors are needed in order for a cell to produce progesterone through the implemented pathway. Therefore, by increasing the concentration of available NADP(H) and NAD(H) co-factors in the cell, it should be theoretically possible to increase the yield of progesterone. 
This can be done using the “CofactorSwapOptimization” algorithm where possible target reactions to swap co-factors can be found.
The obvious choice would be to increase available NAD(H) and NADP(H) on the cost of FAD(H2). However, no FADH2 in the cytosol is included in the model which makes this approach impossible. Therefore, we instead used the algorithm to increase the concentration of available NADP(H) on the cost of NAD(H). Since nine NADP(H) and only one NAD(H) are used in the implemented pathway, this approach should also theoretically increase the yield of progesterone.

In the code the following was done:
- Model iMM904_progesterone was loaded
- A demand for progesterone was made 
- The objective was set to maximize the production of progesterone
- The theoretical yield was set to production of progesterone divided by the optake of glucose
- The algorithm was run and the result was visualized

In [455]:
# Load libraries
import numpy as np
from cobra.io import read_sbml_model, write_sbml_model
from cobra.util import create_stoichiometric_matrix
from cobra import Reaction, Metabolite
from cobra.core.gene import GPR
from cameo.strain_design.heuristic.evolutionary_based import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.optimization import CofactorSwapOptimization
from cameo.strain_design.heuristic.evolutionary.objective_functions import product_yield, biomass_product_coupled_yield
from cameo.util import TimeMachine
from cameo.flux_analysis.analysis import flux_variability_analysis as fva
from cameo import load_model

In [456]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [457]:
# Add demand reaction for progesterone
model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
print(f'Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Adding reaction progesterone demand: progesterone_c --> 


In [458]:
# Choose the reaction that produced progesterone as the objective.
objective_reaction = model.reactions.R02216
model.objective = objective_reaction
print(f'Objective set to maximize {objective_reaction.id}: {objective_reaction.reaction}')

Objective set to maximize R02216: nad_c + pregnenolone_c <=> h_c + nadh_c + progesterone_c


In [459]:
# Setting the progesterone yield as the formation of progesterone divided by the optake of glucose
py = product_yield(model.reactions.R02216, model.exchanges.EX_glc__D_e)
py

In [460]:
# Running the CofactorSwapOptimization Algorithm
# Find reactions that all have the targeted co-factor pairs and add reactions that have the co-factors swapped.
swap = CofactorSwapOptimization(model=model, cofactor_id_swaps=(["nadp_c","nadph_c"],["nad_c","nadh_c"]),objective_function=py)
print(f'Co-factor swap optimization of the model {model.id} used to increase available {swap._metadata["swap_pairs"][0][0]} and {swap._metadata["swap_pairs"][0][1]} on the cost of {swap._metadata["swap_pairs"][1][0]} and {swap._metadata["swap_pairs"][1][1]}')
solution_1 = swap.run(max_size = 10)

Co-factor swap optimization of the model iMM904_progesterone used to increase available nadp_c and nadph_c on the cost of nad_c and nadh_c
Starting optimization at Wed, 23 Nov 2022 15:00:13


HBox()

Finished after 00:03:52


In [461]:
# Printing results
solution_1

,index,targets,fitness
0,0,"(GAPD,)",0.323077
1,1,"(ALCD2x_copy1,)",0.323077
2,2,"(MTHFD, MTHFD2i)",0.323077
3,4,"(MDH, HMGCOAR)",0.323077
4,8,"(ALCD2x_copy2,)",0.323077
5,13,"(MDH, C3STDH2, R01456)",0.262500
6,14,"(MDH, C3STDH2, C3STKR1)",0.262500
7,15,"(HMGCOAR,)",0.253012
8,17,"(MDH,)",0.253012
9,23,"(SQLS, ECYP11A1, C3STKR2, C3STDH2)",0.240000


The reactions listed above can be used as targets for co-factor swapping to increase the theoretical yield of progesterone. The reaction with the highest fitness is GAPD which corospond with finding by King, Zachary A. and Adam M. Feist. This gene was therefore investigated further: 

In [462]:
# Stats of the GAPD reaction
model.reactions.GAPD

Reaction identifier,GAPD
Name,Glyceraldehyde-3-phosphate dehydrogenase
Memory address,0x07f5c219fd3a0
Stoichiometry,g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c Glyceraldehyde 3-phosphate + Nicotinamide adenine dinucleotide + Phosphate <=> 3-Phospho-D-glyceroyl phosphate + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,YJL052W or YJR009C or YGR192C
Lower bound,-999999.0
Upper bound,999999.0


NAP(+) is converted into NADH in the GAPD reaction, however, by swapping the co-factor into NADP(H) it should be possible to get a higher concentration of available NADPH in the cell and thereby increasing the yield of progesterone. As seen above three genes in the model are able to make the GAPD. Thus, to be able to make co-factor swapping these genes have to be knocked out and a NADP(+) dependent GAPD needs to be implemented in the model. This is done in the following code: 

In [463]:
# Loading model
infilename = 'models/iMM904_progesterone.xml'
print(f"Loading {infilename}")
model = load_model(infilename)

Loading models/iMM904_progesterone.xml


In [464]:
# Knock out of GAPD genes
KOs = ["YJL052W","YJR009C","YGR192C"]
model_coswap = model.copy()
model_coswap.id = "iMM904_progesterone_coswap"
print(f"Model {model_coswap.id} were made as a copy of {model.id}")
print(f"For model {model_coswap.id}:")
for KO in KOs:
    model_coswap.genes.get_by_id(KO).knock_out()
    print(f"{KO} were knocked out")

Model iMM904_progesterone_coswap were made as a copy of iMM904_progesterone
For model iMM904_progesterone_coswap:
YJL052W were knocked out
YJR009C were knocked out
YGR192C were knocked out


In [465]:
# Adding NADP dependent GAPD reaction
print(f"For model {model_coswap.id}:")
with open("data/co-factor_swap_reactions.csv","r") as infile:
    infile.readline()
    for line in infile:
        line = line.rstrip().split(",")
        r = Reaction(line[0])
        r.name = line[2]
        if line[3] != "":
            r.subsystem = line[3]
        r.lower_bound = float(line[4])
        r.upper_bound = float(line[5])
        if line[1] != "":
            r.gpr = GPR.from_string(line[1])
        for i in range(int(len(line[6:])/2)):
            metaboliteID = line[6+i*2]
            if metaboliteID != "":
                metaboliteID = model_coswap.metabolites.get_by_id(metaboliteID)
                bin = float(line[7+i*2])
                r.add_metabolites({metaboliteID:bin})
        print(f'Adding reaction {r} | enzyme {r.gpr}')
        model_coswap.add_reactions([r])

For model iMM904_progesterone_coswap:
Adding reaction R01063: g3p_c + nadp_c + pi_c <=> 13dpg_c + h_c + nadph_c | enzyme GDP1


In [466]:
# Add demand reaction for progesterone
models = [model,model_coswap]
for model in models:
    model.add_boundary(model.metabolites.get_by_id('progesterone_c'), type='demand')
    print(f'Model {model.id}: Adding reaction {model.reactions.DM_progesterone_c.name}: {model.reactions.DM_progesterone_c.reaction}')

Model iMM904_progesterone: Adding reaction progesterone demand: progesterone_c --> 
Model iMM904_progesterone_coswap: Adding reaction progesterone demand: progesterone_c --> 


In [467]:
# Calculate µ
biomass = []
for i, model in enumerate(models):
    biomass.append(model.optimize().objective_value)
    print(f'{model.id}: µ = {biomass[i]}')


iMM904_progesterone: µ = 0.2878657037040161
iMM904_progesterone_coswap: µ = 0.29373393760455596


In [468]:
# Calculate flux towards progesterone
pp = []
for i, model in enumerate(models):
    with model:
        objective_reaction = model.reactions.R02216
        model.objective = objective_reaction
        pp.append(model.optimize())
    print(f'{model.id}: progesterone flux = {pp[i].objective_value}')


iMM904_progesterone_coswap: progesterone flux = 0.14285714285714285
iMM904_progesterone_coswap_coswap: progesterone flux = 0.14285714285714285


In [469]:
# Calculate procentage increase in µ
print(f"Procentage increase in µ: {round((biomass[1]/biomass[0]-1)*100,2)}%")

Procentage increase in µ: 2.04%


From the results it is clear that the flux towards progesterone does not change by the co-factor swapping, however, the growth rate does.

In [470]:
# Saving new model
outfilename = "models/iMM904_progesterone_coswap.xml"
print(f"Saving to {outfilename}")
write_sbml_model(model_coswap, outfilename)

Saving to models/iMM904_progesterone_coswap.xml


### References
King, Zachary A., and Adam M. Feist. "Optimal cofactor swapping can increase the theoretical yield for chemical production in Escherichia coli and Saccharomyces cerevisiae." Metabolic Engineering 24 (2014): 117-128.